In [1]:
using Clapeyron
using Plots
using CSV
using DataFrames
using LaTeXStrings
using Measures
using Metaheuristics

In [15]:
model = SAFTgammaMie([("MEA",["NH2"=>1, "CH2"=>1, "CH2OH"=>1]),("Water",["H2O"=>1]),("Sulphur Dioxide",["SO2"=>1])];

        userlocations = (Mw = [16.02285, 14.02658, 31.03392, 18.01528, 64.066],

        epsilon = [284.78 348.39 528.21 339.89 453.85;
                   348.39 473.39 423.17 423.63 301.96;
                   528.21 423.17 407.22 358.02 302.89;
                   339.89 423.63 358.02 266.68 239.53;
                   453.85 301.96 302.89 239.53 237.54],
        sigma = [3.2477, 4.8801, 3.4054, 3.0063, 3.1605],
        lambda_a = [6, 6, 6, 6, 6],
        lambda_r = [10.354 14.063 52.305 13.085 12.412;
                    14.063 19.871 21.230 100.00 17.354;
                    52.305 21.230 22.699 19.619 18.511;
                    13.085 100.00 19.619 17.020 16.085;
                    12.412 17.354 18.511 16.085 15.213],
        vst = [1, 1, 2, 1, 3],
        S = [0.79675, 0.22932, 0.58538, 1, 0.62547],
        n_H=[2, 0, 1, 2, 1],
        n_e=[1, 0, 2, 2, 1],
        epsilon_assoc = Dict([(("NH2","H"),("NH2","e")) => 1070.80,
                              (("NH2","H"),("CH2OH","e")) => 629.88,
                              (("NH2","H"),("H2O","e")) => 1364.40,
                              (("NH2","H"),("SO2","e")) => 663.02,
                              (("CH2OH","H"),("NH2","e")) => 2403.78,
                              (("CH2OH","H"),("CH2OH","e")) => 2097.90,
                              (("CH2OH","H"),("H2O","e")) => 600.00,
                              (("CH2OH","H"),("SO2","e")) => 1415.90,
                              (("H2O","H"),("NH2","e")) => 1877.50,
                              (("H2O","H"),("CH2OH","e")) => 2000.60,
                              (("H2O","H"),("H2O","e")) => 1985.40,
                              (("H2O","H"),("SO2","e")) => 1343.90,
                              (("SO2","H"),("NH2","e")) => 663.02,
                              (("SO2","H"),("CH2OH","e")) => 1415.90,
                              (("SO2","H"),("H2O","e")) => 1343.90,
                              (("SO2","H"),("SO2","e")) => 955.58,
                             ]),
        bondvol = Dict([(("NH2","H"),("NH2","e")) => 95.225e-30,
                        (("NH2","H"),("CH2OH","e")) => 346.08e-30,
                        (("NH2","H"),("H2O","e")) => 22.450e-30,
                        (("NH2","H"),("SO2","e")) => 345.890e-30,
                        (("CH2OH","H"),("NH2","e")) => 26.192e-30,
                        (("CH2OH","H"),("CH2OH","e")) => 62.309e-30,
                        (("CH2OH","H"),("H2O","e")) => 490.19e-30,
                        (("CH2OH","H"),("SO2","e")) => 303.270e-30,
                        (("H2O","H"),("NH2","e")) => 459.18e-30,
                        (("H2O","H"),("CH2OH","e")) => 130.02e-30,
                        (("H2O","H"),("H2O","e")) => 101.69e-30,
                        (("H2O","H"),("SO2","e")) => 353.41e-30,
                        (("SO2","H"),("NH2","e")) => 345.890e-30,
                        (("SO2","H"),("CH2OH","e")) => 303.270e-30,
                        (("SO2","H"),("H2O","e")) => 353.41e-30,
                        (("SO2","H"),("SO2","e")) => 849.99e-30,

                       ])));

In [3]:
@doc """
Inputs:
    model::EoSModel
    T = 292, 313, 333 / K
    w = wt% MEA
    loading = SO2 loading

Outputs:
    partial pressure of SO2 at given T, wt % MEA & SO2 loading

"""
function partial_SO2(model::EoSModel, T, w, loading)

    mw_MEA = 61.08
    mw_H2O = 18.01528

    wt_MEA = w
    wt_H2O = 1 - w

    x_MEA = (wt_MEA / mw_MEA) * (1 / ((wt_MEA / mw_MEA) + (wt_H2O / mw_H2O)))
    x_H2O = 1 - x_MEA
    solvrat = x_H2O / x_MEA

    x_MEA = 1 / (1 + loading + solvrat)
    x_SO2 = x_MEA * loading
    x_H2O = 1 - x_MEA - x_SO2

    x_bub = [x_MEA, x_H2O, x_SO2]

    calc = bubble_pressure(model, T, x_bub)
    pvap = calc[1]
    y_MEA = calc[4][1]
    y_H2O = calc[4][2]
    y_SO2 = calc[4][3]

    out_partial = y_SO2 * pvap
    out_partial = log(out_partial)
    return out_partial
end 


partial_SO2

In [17]:
function error_calc(model, data)
    
    N                    = size(data)[1]
    T                    = data[:, 1]
    w                    = data[:, 2]
    loading              = data[:, 3]
    exp_partial_pressure = data[:, 4]

    partial_pred = zeros(N, 1)
    abs_diff = zeros(N, 1)
    
    for i in 1:N
        partial_pred[i] = partial_SO2(model, T[i], w[i], loading[i])
        abs_diff[i] = abs(partial_pred[i] - (exp_partial_pressure[i]))
    end
    accum_abs_diff = sum(abs_diff)
    accum_abs_diff2 = sum(abs_diff./exp_partial_pressure)
    apd = accum_abs_diff2 * (100/N)
    return apd
end

error_calc (generic function with 1 method)

In [12]:
function get_df(csv_str)
    df = CSV.read(csv_str, DataFrame)
        df_filt = df[3:end, :]
        for i in 1:size(df_filt)[2]
            df_filt[!, i] .= parse.(Float64, df_filt[!, i])
        end 
        df_filt[!, 4] = log.(df_filt[:, 4])
        return df_filt
    end

get_df (generic function with 1 method)

In [13]:
df = get_df("SO2_EXP_All_Data.csv")
df

Row,Clapeyron Estimator,Column2,Column3,Column4
,Float64,Float64,Float64,Float64
1,293.0,0.1,0.0341914,9.49469
2,293.0,0.1,0.0463291,10.1878
3,293.0,0.1,0.0683923,10.5933
4,293.0,0.1,0.108695,10.8809
5,293.0,0.1,0.122807,11.1265
6,313.0,0.1,0.0285087,9.49469
7,313.0,0.1,0.0386155,10.1878
8,313.0,0.1,0.0570175,10.5933
9,313.0,0.1,0.0906749,10.8809


In [18]:
apd = error_calc(model, df)
print(apd)

4.244800067694826